This notebook goes over the basics of neural networks through implementing a multi-layered fully-connected neural network from scratch.

Things to cover:
- automatic gradient computation
- vectorization and matrix multiplication
- back-propagation
- training

In [45]:
import numpy as np

define the building blocks of neural network:
- linear fully-connected layers
- loss computation

each layer and loss component implement the forward propagation step and gradient computation for back propagation.

#### back-propagation
chain rule: https://web.williams.edu/Mathematics/lg5/A37W12/Chain.pdf

the neural network is formed in a layer-by-layer composition of functions. For instance, assuming the input is x0, and the first layer is f(x;W0,b0), we have

$x0 - f0 \rightarrow x1=f0(x0;\theta_0) \rightarrow x2=f1(x1;\theta_1) \rightarrow x3=f2(x2;\theta_2) \rightarrow ...$

Hence using chain rule, we have:
$$\frac{\partial x_{i}}{\partial x_{i-2}}=\frac{\partial f_{i-1}(x_{i-1};\theta_{i-1})}{\partial x_{i-2}}
=
\frac{\partial f_{i-1}(f_{i-2}(x_{i-2};\theta_{i-2});\theta_{i-1})}{\partial x_{i-2}}
=\\
\frac{\partial f_{i-1}(f_{i-2}(x_{i-2};\theta_{i-2});\theta_{i-1})}{\partial f_{i-2}(x_{i-2};\theta_{i-2})}\frac{\partial f_{i-2}(x_{i-2};\theta_{i-2})}{\partial x_{i-2}}
+
\frac{\partial f_{i-1}(f_{i-2}(x_{i-2};\theta_{i-2});\theta_{i-1})}{\partial\theta_{i-1}}\frac{\partial\theta_{i-1}}{\partial x_{i-2}}\\
=\frac{\partial f_{i-1}(x_{i-1};\theta_{i-1})}{\partial x_{i-1}}\frac{\partial x_{i-1}}{\partial x_{i-2}}
+
\frac{\partial f_{i-1}(x_{i-1};\theta_{i-1})}{\partial\theta_{i-1}}\frac{\partial\theta_{i-1}}{\partial x_{i-2}}\\
=
\frac{\partial f_{i-1}(x_{i-1};\theta_{i-1})}{\partial x_{i-1}}\frac{\partial x_{i-1}}{\partial x_{i-2}}\\
$$

Another way to see it is that if $y=f(a_1,a_2,...,a_N)$ is only a function of $a_1$, $\dots$, $a_N$, then we have
$$\frac{\partial y}{\partial b}=\sum_i\frac{\partial y}{\partial a_i}\frac{\partial a_i}{\partial b}$$

Hence we have
$$\frac{\partial x_N}{\partial \theta_i} = \frac{\partial x_N}{\partial x_{i+1}}\frac{\partial x_{i+1}}{\partial \theta_i},$$
$$\frac{\partial x_N}{\partial x_i} = \frac{\partial x_N}{\partial x_{i+1}}\frac{\partial x_{i+1}}{\partial x_i}$$

In [ ]:
class FCLayer:
    def __init__(self, in_size, out_size):
        """
        define the neural network layer by the input size and the output size.
        TODO: For generealization, we can use a general large parameter matrix theta as a
        replacement for W and b.
        """
        W = np.random.random((out_size, in_size)) # weight shape: out_size x in_size
        b = np.random.random((out_size))
        self.out_size = out_size
        self.in_size = in_size
        self.param = [W, b]

    def forward(self, x):
        """
        given an input x of size m x n, where m denotes the batch size, and n is the input size.
        compute an output by:
        W x + b
        of shape m x out_size
        """
        W, b = self.param
        return W.dot(x.T).T + b.reshape((1,-1))
    
    def compute_gradient(self, x):
        """
        compute the jacobian/gradient of the output w.r.t. the weight with
        the input x of shape: m x n
        grad shape: m x out_size x out_size x in_size        
        output: (W_grad, b_grad)
        dy_i/dW_{i,j} = x_j
        dy_i/db_{i} = 1
        dy_i/dx_j = W[i,j]
        """
        W, b =  self.param
        W_grad = np.zeros((len(x), self.out_size, self.out_size, self.in_size))
        b_grad = np.zeros((len(x), self.out_size, self.out_size))
        # use broadcasting for diagonal slices
        idx = np.arange(self.out_size)
        W_grad[:,idx,idx,:] += x.reshape((x.shape[0], 1, x.shape[1]))
        b_grad = b_grad + np.eye(self.out_size).reshape((1, self.out_size, self.out_size))
        x_grad = np.array(W)
        return (W_grad, b_grad), x_grad

    def gradient_descent_step(self, param_grad, lr=1e-3):
        """
        given parameter gradient (same size as the weight), perform one gradient descent step.
        """
        self.param[0] -= lr * param_grad[0]
        self.param[1] -= lr * param_grad[1]


    def compute_gradient_numerical(self, x, eps=1e-4):
        """
        numerical way to compute the gradient for verification of the correctness of the implementation
        dy_i/dW_{j,k} = dy(W+eps)-d(W-eps)/2eps
        """
        W, b =  self.param
        W = np.array(W)
        b = np.array(b)

        W_grad = np.zeros((len(x), self.out_size, self.out_size, self.in_size))
        b_grad = np.zeros((len(x), self.out_size, self.out_size))
        # compute the gradient by perturbing each element
        for j in range(len(W)):
            for k in range(len(W[j])):
                # perturbing W[j,k]
                W[j,k] += eps
                y_plus = W.dot(x.T).T + b.reshape((1,-1))
                W[j,k] -= 2*eps
                y_minus = W.dot(x.T).T + b.reshape((1,-1))
                W_grad[:,:,j,k] = (y_plus - y_minus) / (2*eps)
                W[j,k] += eps  # restore

        for j in range(self.out_size):
            b[j] += eps
            y_plus = W.dot(x.T).T + b.reshape((1,-1))
            b[j] -= 2*eps
            y_minus = W.dot(x.T).T + b.reshape((1,-1))
            b_grad[:,:,j] = (y_plus - y_minus) / (2*eps)
            b[j] += eps  # restore
        
        # verify derivative w.r.t. x
        x_temp = np.array(x)
        x_grad = np.zeros((x.shape[0], self.out_size, self.in_size))
        for i in range(len(x)):
            for j in range(len(x[i])):
                x_temp[i,j] += eps
                y_plus = W.dot(x_temp.T).T + b.reshape((1,-1))
                x_temp[i,j] -= 2*eps
                y_minus = W.dot(x_temp.T).T + b.reshape((1,-1))
                x_grad[i,:,j] = (y_plus - y_minus)[i] / (2*eps)
                x_temp[i,j] += eps  # restore
        return (W_grad, b_grad), x_grad

In [48]:
# test code for verifying FCLayer
in_size = 5
out_size = 7
fcLayer = FCLayer(in_size, out_size)
W, b = fcLayer.param
print('W: ')
print(W)
print('b: ')
print(b)


x = np.random.random((10, in_size))
print('x: ', x)
y = fcLayer.forward(x)
print('y: ', y)
print('y.shape: ', y.shape)
# verify if the output is correct using for-loop (for verifying against vectorization)
for i in range(len(x)):
    W, b = fcLayer.param
    yi = W.dot(x[i]) + b
    print('yi.shape: ', yi.shape)
    print('yi: ')
    print(yi)
    print('y[i]: ')
    print(y[i])
    assert(np.isclose(y[i], yi).all())

grad, x_grad = fcLayer.compute_gradient(x)
W_grad, b_grad = grad
print('x_grad: ')
print(x_grad)

print('W_grad: ')
print(W_grad)
print('b_grad: ')
print(b_grad)

grad_num, x_grad_num = fcLayer.compute_gradient_numerical(x)
W_grad_num, b_grad_num = grad_num
# compute the gradient using numerical method
W_grad_diff = np.linalg.norm(W_grad_num - W_grad)
b_grad_diff = np.linalg.norm(b_grad_num - b_grad)
x_grad_diff = np.linalg.norm(x_grad_num - x_grad)
print('W_grad_diff: ', W_grad_diff)
print('b_grad_diff: ', b_grad_diff)
print('x_grad_diff: ', x_grad_diff)

W: 
[[0.10058577 0.32548229 0.02976696 0.97243409 0.06908887]
 [0.1766955  0.07609467 0.76817591 0.16623673 0.07035571]
 [0.73648811 0.76421214 0.47227127 0.99681991 0.32714391]
 [0.74076242 0.96439735 0.63214915 0.23634061 0.78605033]
 [0.85923821 0.33299582 0.12543781 0.51797779 0.84888386]
 [0.76250184 0.00937683 0.21214427 0.65509099 0.8247184 ]
 [0.10338014 0.80944271 0.26646695 0.80830126 0.74270154]]
b: 
[0.89189944 0.29548965 0.96117748 0.49368736 0.2612125  0.44230609
 0.41797309]
x:  [[0.0135492  0.01462823 0.42831771 0.77076787 0.33118674]
 [0.72546542 0.44703816 0.0339077  0.44777169 0.27559822]
 [0.94686325 0.44903629 0.61489482 0.87060749 0.5250232 ]
 [0.79130017 0.83218478 0.27380409 0.93883728 0.44198019]
 [0.81551011 0.27636687 0.86062234 0.48192744 0.56452607]
 [0.59472073 0.23820507 0.76165278 0.54484894 0.91110554]
 [0.29153494 0.2879805  0.93579459 0.30564863 0.77387223]
 [0.69341342 0.20472357 0.68202462 0.68492385 0.69714438]
 [0.36087541 0.4728263  0.95531433 0.

In [ ]:
class ReLU:
    def __init__(self, in_size):
        """
        define the neural network layer by the input size and the output size.
        TODO: For generealization, we can use a general large parameter matrix theta as a
        replacement for W and b.
        """
        self.in_size = in_size
        self.out_size = out_size
        self.param = []

    def forward(self, x):
        """
        g(x[i]) = 0 if x[i] <= 0; otherwise x[i]
        """
        res = np.array(x)
        res[res<=0] = 0.0
        return res
    
    def compute_gradient(self, x):
        """
        dy_i/dx_j = 0 if i != j
        dy_i/dx_i = 0 if x_i <= 0
        dy_i/dx_i = 1 if x_i > 0
        the input x of shape: m x n
        grad w.r.t. x shape: m x n x n
        """
        x_grad = np.zeros((x.shape[0], x.shape[1], x.shape[1]))
        # use broadcasting for diagonal slices
        idx = np.arange(self.in_size)
        x_grad[:,idx,idx] = (x > 0).astype(float)        
        return (), x_grad

    def gradient_descent_step(self, param_grad, lr=1e-3):
        """
        given parameter gradient (same size as the weight), perform one gradient descent step.
        """
        pass


    def compute_gradient_numerical(self, x, eps=1e-4):
        """
        (y(x+eps)-y(x-eps))/2eps
        shape: m x n x n
        """
        x_grad = np.zeros((x.shape[0], x.shape[1], x.shape[1]))
        x_temp = np.array(x)
        for i in range(len(x)):
            for j in range(len(x[i])):
                x_temp[i,j] += eps
                y_plus = self.forward(x_temp)
                x_temp[i,j] -= 2*eps
                y_minus = self.forward(x_temp)
                x_grad[i,:,j] = (y_plus - y_minus)[i] / (2*eps)
                x_temp[i,j] += eps # restore
        return (), x_grad

In [50]:
# test code for verifying ReLU layer
in_size = 5
relu = ReLU(in_size)

x = np.random.random((10, in_size)) - 0.5
print('x: ', x)
y = relu.forward(x)
print('y: ', y)
print('y.shape: ', y.shape)
# verify if the output is correct using for-loop (for verifying against vectorization)

grad, x_grad = relu.compute_gradient(x)
print('x_grad: ')
print(x_grad)

grad_num, x_grad_num = relu.compute_gradient_numerical(x)
# compute the gradient using numerical method
W_grad_diff = np.linalg.norm(W_grad_num - W_grad)
b_grad_diff = np.linalg.norm(b_grad_num - b_grad)
x_grad_diff = np.linalg.norm(x_grad_num - x_grad)
print('W_grad_diff: ', W_grad_diff)
print('b_grad_diff: ', b_grad_diff)
print('x_grad_diff: ', x_grad_diff)

x:  [[ 0.13228786 -0.31128811  0.05438585 -0.11856732 -0.44988296]
 [-0.32932097  0.17271092  0.31953201 -0.36058933  0.42877293]
 [ 0.14822497  0.24614302 -0.19479945 -0.49159779  0.25312682]
 [ 0.02473314  0.31214884  0.07660131 -0.14586548 -0.28715797]
 [-0.03218097  0.48306264 -0.48967294  0.17092911  0.40598261]
 [ 0.23741837  0.40755568 -0.31595947  0.03794493 -0.43146524]
 [-0.36902465  0.1736176   0.41128677 -0.30725618 -0.09007552]
 [-0.16019618 -0.1410945   0.33636295 -0.0977237   0.4743587 ]
 [-0.00526242  0.22304308  0.39249721 -0.11239164  0.30209213]
 [ 0.06049701  0.22630192 -0.26048769 -0.38813916  0.18775007]]
y:  [[0.13228786 0.         0.05438585 0.         0.        ]
 [0.         0.17271092 0.31953201 0.         0.42877293]
 [0.14822497 0.24614302 0.         0.         0.25312682]
 [0.02473314 0.31214884 0.07660131 0.         0.        ]
 [0.         0.48306264 0.         0.17092911 0.40598261]
 [0.23741837 0.40755568 0.         0.03794493 0.        ]
 [0.         

#### define a neural network model
##### implement the forward propagation
##### implement the back-propagation
store the values of each layer (input and output), chaining the back-propagated jacobian/gradient

In [ ]:
class NeuralNetwrokModel:
    def __init__(self, layers):
        """
        neural network is defined to be a list of layers from the first layer to the last.
        each layer can be (in our simplified case) either a fully-connected layer or ReLU layer.
        """
        input_size = layers[0].in_size
        output_size = layers[-1].out_size
        self.input_size = input_size
        self.output_size = output_size
        self.layers = layers

    def forward(self, x):
        """
        given an input x of size mxn, forward propagate the model.
        """
        for i in range(len(self.layers)):
            x = self.layers[i].forward(x)
        return x

    def compute_gradient(self, x):
        """
        given an input x of size mxn, back-propagate the model to compute the gradients for each parameters.
        - forward once to get the values for each layer (input, output)
        - keep track of d(output)/dxi for each layer

        dout/dxi = dout/dx_{i+1} * dx{i+1}/dxi
        dout/dWi = dout/dx_{i+1} * dx{i+1}/dWi
        
        Caveat:
        dout/dxi is of shape m x n_out x n_in
        dout/dWi is of shape m x n_out x n_out x n_in  (which can use np.matmul to get the result)
        """
        inputs = [x]  # store inputs of each layer
        for i in range(len(self.layers)):
            x = self.layers[i].forward(x)
            inputs.append(x)
        # initialize the jacobian/gradient
        dout_dx = np.zeros((len(x),self.output_size, self.output_size)) + np.eye(self.output_size).reshape((1, self.output_size, self.output_size))
        dout_dparams = []
        for i in range(len(self.layers)-1,-1,-1):
            # compute the commulative value
            param_grad, x_grad = self.layers[i].compute_gradient(inputs[i])
            # dout_dparam = [np.matmul(dout_dx, param_grad[k]) for k in range(len(param_grad))]
            # Method 1: flatten the last few dimensions, and then reshape them
            # for k in range(len(param_grad)):
            #     # dout_dx of shape: N x n_out x n_out_i
            #     # param_grad of shape: N x n_out_i x ...
            #     # for easy compute, flatten the last few dims of the param, conduct matmul, and then reshape
            #     param_grad_k = param_grad[k].reshape((param_grad.shape[0],param_grad.shape[1],-1))
            #     param_grad_k = np.matmul(dout_dx, param_grad_k)
            #     param_grad_k = param_grad_k.reshape(list(dout_dx.shape)[:2] + list(param_grad[k].shape)[2:])
            #     dout_dparam.append(param_grad_k)
            # Method 2: using einsum
        #     # dout_dx of shape: N x n_out x n_out_i
        #     # param_grad of shape: N x n_out_i x ...
            dout_dparam = [np.einsum('ijk...,ik...->ij...', dout_dx, param_grad[k]) for k in range(len(param_grad))]

            dout_dparams.append(dout_dparam)
            dout_dx = np.matmul(dout_dx, x_grad)
            # # DEBUG
            # print(f'layer {i} parameter size:')
            # print(f'input size: {self.layers[i].in_size}, output size: {self.layers[i].out_size}')
            # for param in dout_dparam:
            #     print(param.shape)
            # print('dout_dx shape: ', dout_dx.shape)

        dout_dparams = dout_dparams[::-1]
        return dout_dparams

    def gradient_descent_step(self, dout_dparams, lr=1e-3):
        """
        given parameter gradient (same size as the weight), perform one gradient descent step.
        """
        for i in range(len(dout_dparams)):
            self.layers[i].gradient_descent_step(dout_dparams[i], lr)

    def compute_gradient_numerical(self, x, eps=1e-4):
        """
        perturb each weight value of the neural networks
        compute new forwarded output value, and compute doutput/dw
        shape: N x N_output x weight_shape
        """
        dout_dparams = []
        # loop through the layers to purterb each layer's weights
        for i in range(len(self.layers)):
            params = self.layers[i].param
            param_grad = []
            for param in params:
                # shape of the gradient: N x N_output x weight_shape
                grad = np.zeros([len(x), self.output_size] + list(param.shape))
                # update the weight, and compute
                with np.nditer(param, op_flags=['readwrite'], flags=['multi_index']) as it:
                    for element in it:
                        current_idx = it.multi_index
                        prev_val = np.array(element)
                        element[...] += eps
                        y_plus = self.forward(x)
                        element[...] -= 2*eps
                        y_minus = self.forward(x)
                        idx_tuple = tuple([slice(None), slice(None)] + list(current_idx))
                        grad[idx_tuple] = (y_plus-y_minus)/(2*eps)
                        element[...] = prev_val
                param_grad.append(grad)
            dout_dparams.append(param_grad)
        return dout_dparams

In [63]:
# test the neural network model
input_size = 2
out_size1 = 3
fcl1 = FCLayer(input_size, out_size1)

layers = [fcl1]
nn_model = NeuralNetwrokModel(layers)

x = np.random.random((1, input_size)) - 0.5
y = nn_model.forward(x)
print('y: ')
print(y)

dout_dparams = nn_model.compute_gradient(x)
param_grad, x_grad = fcl1.compute_gradient(x)
print('param_grad: ')
print(param_grad)
print('nn_grad: ')
print(dout_dparams)

dout_dparams_num = nn_model.compute_gradient_numerical(x)
print('numerical: ')
print(dout_dparams_num)


y: 
[[0.65969539 0.18502667 0.01199677]]
layer 0 parameter size:
input size: 2, output size: 3
(1, 3, 3, 2)
(1, 3, 3)
dout_dx shape:  (1, 3, 2)
param_grad: 
(array([[[[ 0.28592869, -0.48186442],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.28592869, -0.48186442],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.28592869, -0.48186442]]]]), array([[[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]]))
nn_grad: 
[[array([[[[ 0.28592869, -0.48186442],
         [ 0.        ,  0.        ],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.28592869, -0.48186442],
         [ 0.        ,  0.        ]],

        [[ 0.        ,  0.        ],
         [ 0.        ,  0.        ],
         [ 0.28592869, -0.48186442]]]]), array([[[1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.]]])]]
numerical

In [66]:
# test the neural network model
input_size = 2
out_size1 = 3
fcl1 = FCLayer(input_size, out_size1)
out_size2 = 4
fcl2 = FCLayer(out_size1, out_size2)

layers = [fcl1, fcl2]
nn_model = NeuralNetwrokModel(layers)


x = np.random.random((10, input_size)) - 0.5
y = nn_model.forward(x)
print('y: ')
print(y)

dout_dparams = nn_model.compute_gradient(x)
# print('param_grad: ')
# print(param_grad)

dout_dparams_num = nn_model.compute_gradient_numerical(x)
# print('numerical: ')
# print(dout_dparams_num)

# compare the two params
for i in range(len(dout_dparams)):
    dout_dparam = dout_dparams[i]
    dout_dparam_num = dout_dparams_num[i]
    for j in range(len(dout_dparam)):
        param_grad = dout_dparam[j]
        param_grad_num = dout_dparam_num[j]
        print('param gradient vs numerical gradient: ', np.linalg.norm(param_grad-param_grad_num))

y: 
[[ 1.44395276  0.91942089  1.48690137  1.37619499]
 [ 1.31352228  0.78541616  1.32538177  1.23555411]
 [ 0.87515579  0.31937988  0.82117784  0.79687416]
 [ 0.98966095  0.46504691  0.89379667  0.85948232]
 [ 1.14550553  0.63922422  1.05207478  0.99698778]
 [ 0.35567625 -0.21985698  0.19150726  0.24872038]
 [ 1.73230421  1.23313343  1.80088409  1.64920243]
 [ 1.43851357  0.92954648  1.4413749   1.33620377]
 [ 1.82938258  1.34852002  1.88247368  1.71989716]
 [ 1.41471609  0.89143288  1.44563571  1.34021789]]
layer 1 parameter size:
input size: 3, output size: 4
(10, 4, 4, 3)
(10, 4, 4)
dout_dx shape:  (10, 4, 3)
layer 0 parameter size:
input size: 2, output size: 3
(10, 4, 3, 2)
(10, 4, 3)
dout_dx shape:  (10, 4, 2)
param gradient vs numerical gradient:  9.608429001887627e-12
param gradient vs numerical gradient:  5.943111187168264e-12
param gradient vs numerical gradient:  6.9253730956772795e-12
param gradient vs numerical gradient:  6.965493601656175e-13


#### define the loss and define the machine learning problem

TODO:
- define losses as classes for computation of gradient
- define machine learning problem as a class.

In [ ]:
class CrossEntropyLoss:
    pass
    # def __init__(self, )

In [ ]:
class MSELoss:
    pass

SyntaxError: incomplete input (1198969800.py, line 1)